In [65]:
import numpy as np
import scipy

import time
import os

from itertools import combinations, product
from functools import reduce
import copy

import pennylane as qml
from pennylane.templates.layers import RandomLayers
# from pennylane import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

from scipy.stats import spearmanr

# import gym

In [50]:
!pip install torch torchquantum

  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchquantum-0.1.4-py3-none-any.whl (119 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached qiskit-0.32.1-py3-none-any.whl
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached qiskit_terra-0.18.3-cp37-cp37m-manylinux2010_x86_64.whl (6.1 MB)
  Using cached qiskit_aqua-0.9.5-py3-none-any.whl (2.1 MB)


In [2]:
!pip install pennylane pennylane-qiskit qiskit matplotlib

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached retworkx-0.11.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached mthree-1.0.0-cp37-cp37m-manylinux

## Train random circuitis

In [ ]:
x_train = np.genfromtxt('./experiment_data/fmnist_4/x_train.txt')
x_test = np.genfromtxt('./experiment_data/fmnist_4/x_test.txt')

x_train = np.concatenate((x_train, x_train), 1)
x_test = np.concatenate((x_test, x_test), 1)

y_train = np.genfromtxt('./experiment_data/fmnist_4/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/fmnist_4/y_test.txt')
# y_test = 1 - 2 * y_test

num_params = [10, 12, 14, 16, 18, 20, 22, 24]

num_qubits = 4
num_embeds = 0

dev = qml.device('lightning.qubit', wires=num_qubits)

for curr_num_params in num_params:
    curr_dir = './random_fe/angle_2/fmnist_4/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    for i in range(5):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)
        
        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, curr_num_params, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
#         circ_gates = ['h'] * num_qubits + ['rz'] * num_qubits + ['zz'] * (num_qubits - 1) + circ_gates
#         gate_params = [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)] + gate_params
#         weights_bounds = [0] * (3 * num_qubits - 1) + weights_bounds
#         inputs_bounds = [0] * (num_qubits + 1) + [i + 1 for i in range(num_qubits)] +\
#             [i + num_qubits + 1 for i in range(num_qubits - 1)] + [2 * num_qubits - 1 for l in range(len(inputs_bounds) - 1)]

        circ_gates = ['rx', 'rx', 'rx', 'rx', 'ry', 'ry', 'ry', 'ry'] * 4 + circ_gates
        gate_params = [[0], [1], [2], [3]] * 8 + gate_params
        weights_bounds = [0 for i in range(32)] + weights_bounds
        inputs_bounds = [i for i in range(32)] + [32 for i in range(len(inputs_bounds))]
        
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                        weights_bounds, [0, 1], 'exp', 'adjoint') 

        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 16000, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list)  

Step 1 | Sliding Loss Window : 1.631777313487817
Step 51 | Sliding Loss Window : 1.4978618009524172
Step 101 | Sliding Loss Window : 1.5246388995940954
Step 151 | Sliding Loss Window : 1.477083616498347
Step 201 | Sliding Loss Window : 1.4620272368524423
Step 251 | Sliding Loss Window : 1.52240457096256
Step 301 | Sliding Loss Window : 1.419959241927482
Step 351 | Sliding Loss Window : 1.4977971871036095
Step 401 | Sliding Loss Window : 1.3170127526578477
Step 451 | Sliding Loss Window : 1.425429330686894
Step 501 | Sliding Loss Window : 1.4500240616970674
Step 551 | Sliding Loss Window : 1.378356644095073
Step 601 | Sliding Loss Window : 1.2840237171248226
Step 651 | Sliding Loss Window : 1.3868874028629639
Step 701 | Sliding Loss Window : 1.303256462639758
Step 751 | Sliding Loss Window : 1.3212016795025965
Step 801 | Sliding Loss Window : 1.335933931011069
Step 851 | Sliding Loss Window : 1.4014536865939429
Step 901 | Sliding Loss Window : 1.344848650315133
Step 951 | Sliding Loss W

## random + fixed embedding

In [ ]:
from datasets import load_dataset

x_train, y_train, x_test, y_test = load_dataset('mnist_2', 'angle', 1)

num_params = [4, 8, 12, 16, 20]

num_qubits = 4
num_embeds = 0

dev = qml.device('lightning.qubit', wires=num_qubits)

for curr_num_params in num_params:
    curr_dir = './random_fe/angle_1/mnist_2/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    for i in range(25):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)
        
        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, curr_num_params, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        circ_gates = ['ry'] * 4 + ['rx'] * 4 + ['rz'] * 4 + ['rx'] * 4 + circ_gates
        gate_params = [[0], [1], [2], [3]] * 4 + gate_params
        inputs_bounds = [j for j in range(16)] + [16 for i in range(len(inputs_bounds))]
        weights_bounds = [0 for i in range(16)] + weights_bounds
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                        weights_bounds, [0], 'exp', 'adjoint') 

        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 8000, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list)   

Step 1 | Sliding Loss Window : 0.33623486751068277
Step 51 | Sliding Loss Window : 1.0650385352524483
Step 101 | Sliding Loss Window : 1.0235950396144957
Step 151 | Sliding Loss Window : 1.0715168578301957
Step 201 | Sliding Loss Window : 1.0181239323620377
Step 251 | Sliding Loss Window : 1.0171473483415192
Step 301 | Sliding Loss Window : 1.0120716487307049
Step 351 | Sliding Loss Window : 0.9059297496452402
Step 401 | Sliding Loss Window : 0.9320571824191026
Step 451 | Sliding Loss Window : 0.8249096520474744
Step 501 | Sliding Loss Window : 1.0078397614428685
Step 551 | Sliding Loss Window : 0.7525105371248028
Step 601 | Sliding Loss Window : 0.9728571445836576
Step 651 | Sliding Loss Window : 1.0263043235698164
Step 701 | Sliding Loss Window : 0.97851684050058
Step 751 | Sliding Loss Window : 0.9181404148710542
Step 801 | Sliding Loss Window : 0.9741850657710345
Step 851 | Sliding Loss Window : 0.8167055026881144
Step 901 | Sliding Loss Window : 0.8621321014553825
Step 951 | Slidi

## train human design circuits

In [ ]:
from datasets import load_dataset
from create_human_design_circs import generate_human_design_circ
from train_circ import mse_loss, mse_vec_loss

import os
import numpy as np

dataset = 'vowel_2'
num_reps = 2

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]

num_params = [16, 20, 24, 28, 32]
num_embed_layers = 1

num_train_steps = 3000
batch_size = 1
loss_fn = mse_loss

ours_dir = './human_design/amp_basic/vowel_2/'

if not os.path.exists(ours_dir):
    os.mkdir(ours_dir)

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train[:, :16].reshape((len(x_train), 1, 16))
x_test = x_test[:, :16].reshape((len(x_test), 1, 16))
# x_train = x_train.reshape((len(x_train), num_embed_layers, num_qubits))
# x_test = x_test.reshape((len(x_test), num_embed_layers, num_qubits))

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RY], [qml.RX]] * 10
enc_layer_options = [[None, True], [None, True]] * 2
# enc_layer_options = [['Y'], ['X']] * 2

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // num_qubits
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'amp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options[:num_embed_layers],
                                          var_layer_options[:curr_num_layers], meas_qubits, 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, num_train_steps, 0.05, batch_size, loss_fn, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([loss_fn(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(np.mean(val_loss))
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 0.5033567237327895
Step 51 | Sliding Loss Window : 1.1495784840689067
Step 101 | Sliding Loss Window : 1.033591124897615
Step 151 | Sliding Loss Window : 1.0220423092829876
Step 201 | Sliding Loss Window : 1.0114098025417706
Step 251 | Sliding Loss Window : 0.9985672968387191
Step 301 | Sliding Loss Window : 0.9713892538349814
Step 351 | Sliding Loss Window : 0.9639057363995767
Step 401 | Sliding Loss Window : 0.8929411477001827
Step 451 | Sliding Loss Window : 0.9519863279605965
Step 501 | Sliding Loss Window : 0.9550800661608307
Step 551 | Sliding Loss Window : 1.0024130544514003
Step 601 | Sliding Loss Window : 0.9396472931305477
Step 651 | Sliding Loss Window : 0.8511393017546787
Step 701 | Sliding Loss Window : 1.049250006760423
Step 751 | Sliding Loss Window : 0.9772310169637227
Step 801 | Sliding Loss Window : 0.9551775535465178
Step 851 | Sliding Loss Window : 0.9093430920553939
Step 901 | Sliding Loss Window : 0.9083461544960443
Step 951 | Slidin

In [43]:
circ = generate_human_design_circ(dev, num_qubits, 'amp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options,
                                          var_layer_options[:curr_num_layers], meas_qubits, 'state')

circ(x_train[0], np.zeros((1, 1, 4)))

<tf.Tensor: shape=(16,), dtype=complex128, numpy=
array([0.18830212+0.j, 0.40829304+0.j, 0.17001273+0.j, 0.13760369+0.j,
       0.13760369+0.j, 0.17001273+0.j, 0.40829304+0.j, 0.18830212+0.j,
       0.18830212+0.j, 0.40829304+0.j, 0.17001273+0.j, 0.13760369+0.j,
       0.13760369+0.j, 0.17001273+0.j, 0.40829304+0.j, 0.18830212+0.j])>

In [3]:
!python supernet/train_search_bank.py --warmup_epochs 0 --steps 1 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 1 --n_encode_layers 1 --save "4_params" --data "./supernet/bank_data" --save_dir "./supernet/bank"

2022-09-28 08:16:12.051768: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-09-28 08:16:12.051882: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-09-28 08:16:12.081126: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(1098, 2, 4) (122, 2, 4)
Traceback (most recent call last):
  File "supernet/train_search_bank.py", line 234, in <module>
    main()
  File "supernet/train_search_bank.py", line 178, in main
    subnet = np.random.randint(0, len(NAS_search_space), (args.n_layers,)).tolist()
NameError: name 'NAS_search_space' is not defined


In [ ]:
!python supernet/train_search_bank.py --warmup_epochs 0 --steps 13725 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 2 --n_encode_layers 1 --save "8_params" --data "./supernet/bank_data" --save_dir "./supernet/bank"

In [ ]:
!python supernet/train_search_bank.py --warmup_epochs 0 --steps 13725 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 3 --n_encode_layers 1 --save "12_params" --data "./supernet/bank_data" --save_dir "./supernet/bank"

In [ ]:
!python supernet/train_search_bank.py --warmup_epochs 0 --steps 13725 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 4 --n_encode_layers 1 --save "16_params" --data "./supernet/bank_data" --save_dir "./supernet/bank"

In [ ]:
!python supernet/train_search_bank.py --warmup_epochs 0 --steps 13725 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 5 --n_encode_layers 1 --save "20_params" --data "./supernet/bank_data" --save_dir "./supernet/bank"

In [ ]:
!python supernet/train_search_fmnist.py --warmup_epochs 0 --steps 50000 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 5 --n_encode_layers 1 --save "20_params" --data "./supernet/fmnist_4_data" --save_dir "./supernet/fmnist_4"

2022-08-06 13:39:05.627799: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-08-06 13:39:05.627905: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-08-06 13:39:05.656878: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(8000, 4, 4) (2000, 4, 4)
subnet: [111599, 250597, 52840, 236018, 274497], expert_idx: 0
2022-08-06 13:39:18.415732: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-06 13:39:18.441845: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequen

## our metric

In [28]:
search_params = [12, 16, 20, 24]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

num_qubits = 4
num_embeds = 0

x_train = np.genfromtxt('./experiment_data/fmnist_4/x_train.txt')[:, :16]
x_test = np.genfromtxt('./experiment_data/fmnist_4/x_test.txt')[:, :16]

y_train = np.genfromtxt('./experiment_data/fmnist_4/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/fmnist_4/y_test.txt')
# y_test = 1 - 2 * y_test

class_0_sel = np.random.choice(2000, 16, False)
class_1_sel = np.random.choice(2000, 16, False) + 2000
class_2_sel = np.random.choice(2000, 16, False) + 4000
class_3_sel = np.random.choice(2000, 16, False) + 6000
sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

sel_data = x_train[sel_inds]

# np.savetxt('./ours/fmnist_4/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = './ours/fmnist_4/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(500):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((32, p))
#         np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
#         np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
#         np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
#         np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
#         np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
    
        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((64, 64))
        circ_t_mean_mat = np.zeros((64, 64))
    
        for j in range(32):
            curr_params = np.concatenate([params[j] for k in range(64)]).reshape((64, p))
            mat = compute_reduced_similarity(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, curr_params, sel_data, [0, 1])
        
            t_mat = mat > ((np.sum(mat) - 64) / 4032)
        
            diff_mat = mat - ideal
            diff_2d = np.sqrt(np.sum(np.multiply(diff_mat, diff_mat)))
            
            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sqrt(np.sum(np.multiply(diff_t_mat, diff_t_mat)))
            
            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)
            
#         np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
#         np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
#     np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
#     np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
#     np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
#     np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
#     np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
#     np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
#     np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
#     np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

ValueError: RZ: wrong number(s) of dimensions in parameters. Parameters with ndims (2,) passed, (0,) expected.

In [13]:
s

NameError: name 's' is not defined

## Our metric + fixed mebdding

In [ ]:
search_params = [2, 4, 6, 8, 10, 12, 14, 16]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 2
num_embeds = 0

x_train = np.genfromtxt('./experiment_data/moons_300/x_train.txt')
x_test = np.genfromtxt('./experiment_data/moons_300/x_test.txt')

y_train = np.genfromtxt('./experiment_data/moons_300/y_train.txt')[:, 0]
y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/moons_300/y_test.txt')[:, 0]
y_test = 1 - 2 * y_test

class_0_sel = np.random.choice(240, 16, False)
class_1_sel = np.random.choice(240, 16, False) + 240
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]

np.savetxt('./ours_fe/moons/angle_2/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = './ours_fe/moons/angle_2/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    for i in range(250):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((32, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        circ_gates = ['ry'] * 4 + circ_gates
        gate_params = [[0], [1]] * 2 + gate_params
        inputs_bounds = [k for k in range(4)] + [4 for i in range(len(inputs_bounds))]
        weights_bounds = [0 for k in range(4)] + weights_bounds        
           
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
    
        circ_d2_scores = []
        circ_d2_t_scores = []
    
        for j in range(32):
            curr_params = np.concatenate([params[j] for k in range(32)]).reshape((32, p))
            mat = compute_reduced_similarity(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - 32) / 996)
        
            diff_mat = mat - ideal
            diff_2d = np.sqrt(np.sum(np.multiply(diff_mat, diff_mat)))
            
            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sqrt(np.sum(np.multiply(diff_t_mat, diff_t_mat)))
            
            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)

In [44]:
from importlib import reload

import create_gate_circs

reload(create_gate_circs)

<module 'create_gate_circs' from '/root/create_gate_circs.py'>

In [ ]:
from datasets import load_dataset
from create_gate_circs import create_batched_gate_circ, generate_random_gate_circ
from metrics import compute_reduced_similarity

search_params = [4, 8, 12, 16, 20]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 4
num_embeds = 8
meas_qubits = [0]

x_train, y_train, x_test, y_test = load_dataset('bank', 'angle', 2)

class_0_sel = np.random.choice(549, 16, False)
class_1_sel = np.random.choice(549, 16, False) + 549
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]
num_data = len(sel_data)

num_param_runs = 32

np.savetxt('./ours/bank/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = './ours/bank/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(2500):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((num_param_runs, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)

        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((num_data, num_data))
        circ_t_mean_mat = np.zeros((len(sel_data), len(sel_data)))

        for j in range(num_param_runs):
            curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / num_param_runs
            circ_t_mean_mat += t_mat / num_param_runs
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
    np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
    np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235


## Train supernet selected circuits

In [26]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'bank'
main_dir = './supernet/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 2)

# inds = np.random.choice(len(x_test), 250, False)
# x_test = x_test[inds]
# y_test = y_test[inds]

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 2

device_name = 'ibmq_lima'
# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [12, 20]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)
#         curr_circ_desc = [[272920], [163863], [258091], [172861], [126492]][i]

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0], 'exp')
        
        val_exp_list = []
        
        for j in range(len(x_test)):
            val_exp_list.append(circ(x_test[j], curr_params))
            
        val_exps = np.array(val_exp_list).flatten()
        val_losses = np.power(val_exps - y_test, 2)
        val_loss = np.mean(val_losses)
        acc = np.mean(val_losses < 1)
    
        print(curr_circ_desc, acc, val_loss, np.sum(val_losses < 1), len(x_test))

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)  
#     np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

ibmqfactory.load_account:WARNING:2022-10-02 13:59:52,765: Credentials are already in use. The existing account in the session will be replaced.


[132179  69367 145983] 0.5655737704918032 1.0025491558137487 69 122
[ 93051 269986 235179] 0.45901639344262296 1.179101975237737 56 122
[231321 268550 260695] 0.6229508196721312 0.9105262130987449 76 122
[  6108 213144 325507] 0.6065573770491803 0.930874590013848 74 122
[  9280 154605 109814] 0.45081967213114754 1.0686466342113057 55 122
[279577  28019 211796] 0.5491803278688525 0.9942925249943968 67 122
[105975   1096 268285] 0.47540983606557374 1.0489581999231556 58 122
[ 72530 292709  88408] 0.4672131147540984 1.0597194921774942 57 122
[ 93925 179169 280692] 0.45901639344262296 1.1754785131235592 56 122
[202502 163168  53305] 0.4426229508196721 1.0767296963050716 54 122
[251102   1705  76097] 0.4344262295081967 1.109802496237833 53 122
[ 69530  88900 143965] 0.5327868852459017 1.015002516449475 65 122
[265101  71322 151143] 0.45901639344262296 1.0686834991955367 56 122
[296048 241796 147336] 0.45901639344262296 1.0551693713078734 56 122
[162980 311203  81629] 0.48360655737704916 1.0

In [ ]:
x_train = np.genfromtxt('./experiment_data/fmnist_4/x_train.txt')[:, :16]
x_test = np.genfromtxt('./experiment_data/fmnist_4/x_test.txt')[:, :16]

prod_1 = np.prod(np.pi - x_train[:, :4], 1).reshape((len(x_train), 1))
prod_2 = np.prod(np.pi - x_train[:, 4:8], 1).reshape((len(x_train), 1))
prod_3 = np.prod(np.pi - x_train[:, 8:12], 1).reshape((len(x_train), 1))
prod_4 = np.prod(np.pi - x_train[:, 12:16], 1).reshape((len(x_train), 1))

x_train = np.concatenate((x_train[:, :4], prod_1, prod_1, prod_1, x_train[:, 4:8], prod_2, prod_2, prod_2, x_train[:, 8:12], prod_3, prod_3, prod_3,
                          x_train[:, 12:16], prod_4, prod_4, prod_4), 1)

prod_1 = np.prod(np.pi - x_test[:, :4], 1).reshape((len(x_test), 1))
prod_2 = np.prod(np.pi - x_test[:, 4:8], 1).reshape((len(x_test), 1))
prod_3 = np.prod(np.pi - x_test[:, 8:12], 1).reshape((len(x_test), 1))
prod_4 = np.prod(np.pi - x_test[:, 12:16], 1).reshape((len(x_test), 1))

x_test = np.concatenate((x_test[:, :4], prod_1, prod_1, prod_1, x_test[:, 4:8], prod_2, prod_2, prod_2, x_test[:, 8:12], prod_3, prod_3, prod_3,
                          x_test[:, 12:16], prod_4, prod_4, prod_4), 1)

y_train = np.genfromtxt('./experiment_data/fmnist_4/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/fmnist_4/y_test.txt')
# y_test = 1 - 2 * y_test

num_qubits = 4
num_encoding_layers = 4

dev = qml.device('lightning.qubit', wires=num_qubits)

# search_nums = [5, 10, 25, 50, 100, 250]
search_nums = [100, 250]
num_trials = [5, 5, 5, 5, 5, 5]
param_nums = [12, 16, 20, 24]

for p in param_nums:
    curr_params_dir =  './supernet/fmnist_4/search-{}_params'.format(p)
    
    sc_circs = np.genfromtxt(curr_params_dir + '/searched_circ_accs.txt', dtype=str, delimiter=']')

    circ_layers = [[int(j) for j in i[0][1:].split(' ') if j.isnumeric()] for i in sc_circs]
    circ_accs = [float(i[1]) for i in sc_circs]
    
    circ_layers = np.array(circ_layers)
    circ_accs = np.array(circ_accs)

    layer_gates = [i[1:].split(')') for i in open(curr_params_dir + '/search_space.txt').read().split('\n')][:-1]

    layer_rots = [''.join([j for j in i[0] if j.isupper()]) for i in layer_gates]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = [''.join([j for j in i[1][1:] if j not in ['[', ']', ',', ' ']]) for i in layer_gates]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/search_{}'.format(search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = np.random.choice(500, search_nums[k], False)
            best_subnet = circ_layers[chosen_circs[np.argmax(circ_accs[chosen_circs])]]

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_encoding_layers,
                                                                                                          layer_rots, layer_cnots, num_qubits)
            
            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                            weights_bounds, [0, 1], 'exp', 'adjoint') 

            losses_list = []
            accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 16000, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                losses_list.append(val_loss)
                accs_list.append(acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)            

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

Step 1 | Sliding Loss Window : 1.494311051750528
Step 51 | Sliding Loss Window : 1.4312575296745027
Step 101 | Sliding Loss Window : 1.4213322949645235
Step 151 | Sliding Loss Window : 1.390058166954078
Step 201 | Sliding Loss Window : 1.4116658445520534
Step 251 | Sliding Loss Window : 1.403141546253656
Step 301 | Sliding Loss Window : 1.4242730365779497
Step 351 | Sliding Loss Window : 1.4205874127549725
Step 401 | Sliding Loss Window : 1.4278182010915588
Step 451 | Sliding Loss Window : 1.4284182393829608
Step 501 | Sliding Loss Window : 1.4262338734831344
Step 551 | Sliding Loss Window : 1.4054655760738093
Step 601 | Sliding Loss Window : 1.4442547102911893
Step 651 | Sliding Loss Window : 1.4112036761851638
Step 701 | Sliding Loss Window : 1.409920900143539
Step 751 | Sliding Loss Window : 1.398428300208449
Step 801 | Sliding Loss Window : 1.3910408941639865
Step 851 | Sliding Loss Window : 1.4322700895248466
Step 901 | Sliding Loss Window : 1.3697168942720708
Step 951 | Sliding L

## train our circuits

In [ ]:
from datasets import load_dataset
from create_gate_circs import get_circ_params, create_gate_circ
from train_circ import train_qnn, mse_loss

search_nums = [100]
param_nums = [2, 4, 6, 8, 10, 12, 14, 16]

dataset = 'moons_300'
num_reps = 1

num_qubits = 2
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]

ours_dir = './ours_fe/moons/iqp_1/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'iqp', num_reps)

for search_num in search_nums:       
    for param_num in param_nums:
        curr_dir = ours_dir + '/{}_params/search_{}'.format(param_num, search_num)
        
        if not os.path.exists(curr_dir):
            os.mkdir(curr_dir)
    
        mean_t_mat_scores = np.genfromtxt(ours_dir + '{}_params/d2_mean_t_mat_scores.txt'.format(param_num))

        for j in range(25):
            curr_trial_dir = curr_dir + '/trial_{}'.format(j + 1)

            if not os.path.exists(curr_trial_dir):
                os.mkdir(curr_trial_dir)
        
            sel_inds = range(j * 100, j * 100 + 100)
            sel_ind = sel_inds[np.argmin(mean_t_mat_scores[sel_inds])]

            np.savetxt(curr_trial_dir + '/searched_circuit_inds.txt', sel_inds)
            np.savetxt(curr_trial_dir + '/searched_circuit_scores.txt', mean_t_mat_scores[sel_inds])
            np.savetxt(curr_trial_dir + '/sel_circuit_ind.txt', [sel_ind])

            circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(ours_dir + '{}_params/circ_{}'.format(param_num, sel_ind + 1))

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint')
        
            losses_list = []
            accs_list = []

            for j in range(5):
                curr_train_dir = curr_trial_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 2400, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                losses_list.append(val_loss)
                accs_list.append(acc)

            np.savetxt(curr_trial_dir + '/accs.txt', accs_list)
            np.savetxt(curr_trial_dir + '/val_losses.txt', losses_list)          

Step 1 | Sliding Loss Window : 1.0
Step 51 | Sliding Loss Window : 1.0
Step 101 | Sliding Loss Window : 1.0
Step 151 | Sliding Loss Window : 1.0
Step 201 | Sliding Loss Window : 1.0
Step 251 | Sliding Loss Window : 1.0
Step 301 | Sliding Loss Window : 1.0
Step 351 | Sliding Loss Window : 1.0
Step 401 | Sliding Loss Window : 1.0
Step 451 | Sliding Loss Window : 1.0
Step 501 | Sliding Loss Window : 1.0
Step 551 | Sliding Loss Window : 1.0
Step 601 | Sliding Loss Window : 1.0
Step 651 | Sliding Loss Window : 1.0
Step 701 | Sliding Loss Window : 1.0
Step 751 | Sliding Loss Window : 1.0
Step 801 | Sliding Loss Window : 1.0
Step 851 | Sliding Loss Window : 1.0
Step 901 | Sliding Loss Window : 1.0
Step 951 | Sliding Loss Window : 1.0
Step 1001 | Sliding Loss Window : 1.0
Step 1051 | Sliding Loss Window : 1.0
Step 1101 | Sliding Loss Window : 1.0
Step 1151 | Sliding Loss Window : 1.0
Step 1201 | Sliding Loss Window : 1.0
Step 1251 | Sliding Loss Window : 1.0
Step 1301 | Sliding Loss Window : 1

## Train correlatuon circuits

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibm_oslo'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(400, 600):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

1.0978491207517789 1.0986833019256592 401


In [9]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_human_design_circs import generate_human_design_circ
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

num_qubits = 4
num_embeds = 16

meas_qubits = [0]

device_name = 'ibmq_lima'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [4, 8, 12, 16, 20]

num_embed_layers = [4, 4, 1]
repetitions = [1, 1, 1]

enc_layer_options = [[['X'], ['Y']] * 2, [[1, None], [1, None]] * 2, [[None, True], [None, True]]]
var_layer_options = [[[qml.RX], [qml.RY]] * 4, [[qml.RX], [qml.RY]] * 4, [[qml.RX], [qml.RY]] * 4]
encoding_types = ['angle',  'iqp', 'amp']

for ec in range(len(encoding_types)):
    x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', repetitions[ec])
    
    x_train = x_train.reshape((len(x_train), num_embed_layers[ec], -1))
    x_test = x_test.reshape((len(x_test), num_embed_layers[ec], -1))
    
    main_dir = './human_design/{}_basic/mnist_2'.format(encoding_types[ec])
    
    for p in param_nums:
        curr_weights_shape = (p // num_qubits, 1, num_qubits)
        
        curr_dir = main_dir + '/{}_params'.format(p)
                     
        circ = generate_human_design_circ(dev, num_qubits, encoding_types[ec], 'basic', num_embed_layers[ec], p // num_qubits, enc_layer_options[ec],
                                          var_layer_options[ec][:p // num_qubits], meas_qubits, 'exp')

        noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

        losses_list = []
        accs_list = []

        curr_dev_dir = curr_dir + '/' + device_name

        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = (np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]).reshape(curr_weights_shape)

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
        
    #             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list))

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)

ibmqfactory.load_account:WARNING:2022-09-19 08:58:10,222: Credentials are already in use. The existing account in the session will be replaced.


0.8111063215357499 0.8090769481658936
0.4729824537749028 0.4993972911834717
0.27143875214908886 0.31375459098815917
0.27138508784778864 0.3194785175323486
0.21348001145415085 0.26610957527160645
0.9946072554517877 0.9958887519836426
0.822643930674137 0.7859517059326172
0.7308705133428212 0.72038551902771
0.738905850647093 0.7445263042449951
0.7198257218072618 0.7222175579071045
0.5941922056080466 0.8978467826843262
0.26794174680529276 0.5998846950531006
0.21063736525835425 0.9989795799255371
0.18421056150835532 1.0470421104431151
0.14933250939415632 1.0480890674591063


In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_vec_loss

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

print(x_train.shape)

num_qubits = 4
num_embeds = 16
num_params = 32

for i in range(550, 660):
    curr_dir = './experiment_data/mnist_4/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0, 1], 'exp')    

    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        
        info = train_qnn(circ, x_train, y_train, x_test, y_test, [num_params], 16000, 0.05, 1, mse_vec_loss, verbosity=127300, 
                                                                                        loss_window=50, init_params=None, 
                                                                                        acc_thres=1.1, shuffle=True, print_loss=50)
        
        val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
        val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
        
        np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
        np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])
        
        losses_list.append(val_loss)
        accs_list.append(acc)
        
    np.savetxt(curr_dir + '/val_losses.txt', losses_list)
    np.savetxt(curr_dir + '/accs.txt', accs_list)

(8000, 16)
Step 1 | Sliding Loss Window : 1.5440169112395021
Step 51 | Sliding Loss Window : 1.4699525099000235
Step 101 | Sliding Loss Window : 1.4229925118343403
Step 151 | Sliding Loss Window : 1.4059230835743168
Step 201 | Sliding Loss Window : 1.3600395623447177
Step 251 | Sliding Loss Window : 1.2916015456263004
Step 301 | Sliding Loss Window : 1.2626864021682365
Step 351 | Sliding Loss Window : 1.2780166088855875
Step 401 | Sliding Loss Window : 1.0443428165630477
Step 451 | Sliding Loss Window : 1.2553989897293805
Step 501 | Sliding Loss Window : 1.2102974834900402
Step 551 | Sliding Loss Window : 1.118092510088995
Step 601 | Sliding Loss Window : 1.1773906937499101
Step 651 | Sliding Loss Window : 1.1930313474957683
Step 701 | Sliding Loss Window : 1.1960291471406579
Step 751 | Sliding Loss Window : 1.203652800950536
Step 801 | Sliding Loss Window : 1.2662433455737032
Step 851 | Sliding Loss Window : 1.2487963665553723
Step 901 | Sliding Loss Window : 1.1400363580642412
Step 9

In [86]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets import TorchDataset
from create_gate_circs import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_data = TorchDataset(dataset, 'angle', 1, True)
test_data = TorchDataset(dataset, 'angle', 1, False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))

val_test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, sampler=torch.utils.data.SequentialSampler(test_data))

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

for i in range(11, 18):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)

    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        accs_history = train_tq_model(model, opt, loss, train_data_loader, test_data_loader, 120000, 1000, 1000)
        
#         curr_val_losses = []
#         curr_acc = 0
        
#         for k in range(100):
#             val_x, val_y = next(iter(val_test_data_loader))

#             val_preds = model(val_x)
#             curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
#             val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()

#             curr_acc += np.mean(val_class == val_y.detach().numpy().flatten())
        
#         np.save(curr_train_dir + '/accs_history_{}.npy'.format(j + 1), accs_history)
        
#         losses_list.append(curr_val_losses)
#         accs_list.append(curr_acc / 100)
        
#         print(curr_acc)
        
    torch.save(model.state_dict(), curr_dir + '/model.pt')
#     np.save(curr_dir + '/val_losses.npy', losses_list)
#     np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Accuracy: 1.0
Step 1001 | Accuracy: 0.0
Step 2001 | Accuracy: 1.0
Step 3001 | Accuracy: 0.0
Step 4001 | Accuracy: 0.0
Step 5001 | Accuracy: 1.0
Step 6001 | Accuracy: 0.0
Step 7001 | Accuracy: 1.0
Step 8001 | Accuracy: 1.0
Step 9001 | Accuracy: 0.0
Step 10001 | Accuracy: 0.0
Step 11001 | Accuracy: 1.0
Step 12001 | Accuracy: 1.0
Step 13001 | Accuracy: 0.0
Step 14001 | Accuracy: 0.0
Step 15001 | Accuracy: 0.0
Step 16001 | Accuracy: 1.0
Step 17001 | Accuracy: 0.0
Step 18001 | Accuracy: 1.0
Step 19001 | Accuracy: 0.0
Step 20001 | Accuracy: 0.0
Step 21001 | Accuracy: 0.0
Step 22001 | Accuracy: 1.0
Step 23001 | Accuracy: 1.0
Step 24001 | Accuracy: 1.0
Step 25001 | Accuracy: 1.0
Step 26001 | Accuracy: 0.0
Step 27001 | Accuracy: 0.0
Step 28001 | Accuracy: 1.0
Step 29001 | Accuracy: 1.0
Step 30001 | Accuracy: 0.0
Step 31001 | Accuracy: 1.0
Step 32001 | Accuracy: 1.0


## noise mtrci for correlaiton circuits

In [57]:
from importlib import reload

import create_gate_circs

reload(create_gate_circs)

<module 'create_gate_circs' from '/root/create_gate_circs.py'>

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_4'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './ours/{}'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

param_nums = [20, 24]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_lima'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for p in param_nums:
    param_dir = curr_dir + '/{}_params'.format(p)
    
    for i in range(2500):
        circ_dir = param_dir + '/circ_{}'.format(i + 1)

        circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

        noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noise_metric_dir = circ_dir + '/noise_metric'

        if not os.path.exists(noise_metric_dir):
            os.mkdir(noise_metric_dir)

        device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

        if not os.path.exists(device_noise_metric_dir):
            os.mkdir(device_noise_metric_dir)

        params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
        batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]

        noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
        noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

        actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))

        tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)

        np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
        np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])

        score_tvds.append(1 - tvd)
        actual_tvds.append(actual_tvd)

        print(1 - tvd, actual_tvd, i)

ibmqfactory.load_account:WARNING:2022-09-26 05:57:31,764: Credentials are already in use. The existing account in the session will be replaced.


0.9214324951171875 0.9208984375 0
0.7428359985351562 0.7738876342773438 1
0.9144668579101562 0.9171218872070312 2
0.9364013671875 0.9360504150390625 3
0.9017105102539062 0.8969345092773438 4
0.9194259643554688 0.9219207763671875 5
0.8599853515625 0.8703460693359375 6
0.9171676635742188 0.9206924438476562 7
0.9005584716796875 0.8808441162109375 8
0.84368896484375 0.8502349853515625 9
0.74200439453125 0.8023147583007812 10
0.9031448364257812 0.9055633544921875 11
0.9029464721679688 0.9086837768554688 12
0.8378677368164062 0.8433837890625 13
0.895721435546875 0.895599365234375 14
0.92169189453125 0.9236526489257812 15
0.7304840087890625 0.7269287109375 16
0.839630126953125 0.846435546875 17
0.9237136840820312 0.920806884765625 18
0.9043350219726562 0.9095001220703125 19
0.8807449340820312 0.876373291015625 20
0.8555221557617188 0.881591796875 21
0.922943115234375 0.9232864379882812 22
0.6365814208984375 0.7033309936523438 23
0.8959426879882812 0.888397216796875 24
0.9052810668945312 0.888

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_2'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_lima'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-11-05 17:02:52,583: Credentials are already in use. The existing account in the session will be replaced.


0.8637542724609375 0.8972930908203125 0
0.80694580078125 0.8306808471679688 1
0.8868026733398438 0.8911514282226562 2
0.8519363403320312 0.8625946044921875 3
0.8143310546875 0.8325424194335938 4
0.8664169311523438 0.871002197265625 5
0.8963775634765625 0.8997802734375 6
0.8989791870117188 0.9125137329101562 7
0.8377685546875 0.8474273681640625 8
0.890045166015625 0.8882217407226562 9
0.7268829345703125 0.7903213500976562 10
0.8538436889648438 0.8706283569335938 11
0.91400146484375 0.9260330200195312 12
0.9017562866210938 0.9049530029296875 13
0.884796142578125 0.8817062377929688 14
0.8644180297851562 0.8602371215820312 15
0.8645248413085938 0.8863677978515625 16
0.846099853515625 0.8472061157226562 17
0.8876724243164062 0.8866729736328125 18
0.8269882202148438 0.8364715576171875 19
0.910614013671875 0.9168319702148438 20
0.9010696411132812 0.9125137329101562 21
0.87164306640625 0.8825225830078125 22
0.8681259155273438 0.8390121459960938 23
0.8832473754882812 0.8959884643554688 24
0.879